In [ ]:
from helper import *
import sys
import importlib
import numpy as np

## Import Metrics
sys.path.append("/home/hj/vis2021_snc/metrics4mdp/metrics4mdp")
from provider import MDPMetricProvider

## Import SNC
sys.path.append("/home/hj/vis2021_snc/steadiness_cohesiveness")
import snc as sc




PATH_TO_DATASET = "../../dataset-generator/dataset/"

In [ ]:
import time
min_dist = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
n_neighbors = [3]

n_spheres = [3, 5, 7, 10, 15, 20]
radius = [1, 3, 5, 7, 9]

# klist=[40, 45, 50, 55, 60, 65, 70]
klist = [40]


## reload SNC for the testing
importlib.reload(sc)
SNC = sc.SNC


def test_umap_dataset_sc(dataset,sample_rate, min_dist, n_neighbors, klist, rad, ns):
    for k in klist:
        steadiness_result = {}
        cohesiveness_result = {}
        for md in min_dist:
            s_temp_result = {}
            c_temp_result = {}
            for nn in n_neighbors:
                mnist_umap_path = make_path(PATH_TO_DATASET, dataset, "umap", sample_rate, [md, nn, rad, ns])
                emb_data, raw_data, label_data = get_data(mnist_umap_path)

                raw_data = np.array(raw_data)
                emb_data = np.array(emb_data)
                label_data = np.array(label_data)

                parameter = {
                    "k": k
                }

                start = time.time()
                snc = SNC(raw_data, emb_data, iteration=300, cluster_parameter = parameter)
                snc.fit()
                s_temp_result[nn] = snc.steadiness()
                c_temp_result[nn] = snc.cohesiveness()
                print(md,nn,"finished")
                print(s_temp_result[nn], c_temp_result[nn])
            steadiness_result[md] = s_temp_result
            cohesiveness_result[md] = c_temp_result
        data = {
            "steadiness": steadiness_result,
            "cohesivenss": cohesiveness_result
        }
        print("finished k =",k)
        import json
        with open("./" + dataset + "_" + str(sample_rate) + "_" + str(k) + "_result.json", "w") as outfile:
            json.dump(data, outfile)

for r in radius:
    for ns in r_spheres:
        test_dataset_sc("spheres", 2, min_dist, n_neighbors, klist, r, ns)